# Classifieur thermique et visuel

In [17]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import os

df = pd.read_csv('base_de_données.csv')
df_train = pd.read_csv('X_train.csv')
df_test = pd.read_csv('X_test.csv')
csv_thermiques_train = df_train['nom_fichier']
indices_train = df_train['index']
csv_thermiques_test = df_test['nom_fichier']
indices_test = df_test['index']
csv_thermiques = df['nom_fichier']
csv = csv_thermiques[0]
csv_0 = (csv.split('.'))[0]
sift_th = cv2.SIFT_create()

In [18]:
images = df['photo']
images_train = df_train['photo']
images_test = df_test['photo']


#sift = cv2.SIFT_create(edgeThreshold=2.4,contrastThreshold=0.011)
sift = cv2.SIFT_create(edgeThreshold=2,contrastThreshold=0.014,sigma=0.5)

Création des descripteurs

In [19]:

buff =  cv2.imread('./Photographies pieces chaudes/' + images[0] ,cv2.IMREAD_GRAYSCALE)

mask = np.zeros_like(buff)
z = np.ones((550,600))
mask[350:900,600:1200] = z
mask = np.uint8(mask)

i=0
for image,csv in zip(images_train,csv_thermiques_train) :
	nom_csv = (csv.split('.'))[0]
	if i==0:
		img_th = cv2.imread('./cv2_thermiques/' + nom_csv + '.png',cv2.IMREAD_GRAYSCALE)
		mask_th = np.load('./mask_thermique/'+ nom_csv + '.npy')
		
		#kp,des = sift.detectAndCompute(img,None)
		kp_th = sift_th.detect(img_th,mask = mask_th)
		kp_th,des_th = sift_th.compute(img_th,kp_th)
		descripteurs_th = des_th
		
		img = cv2.imread('./Photographies pieces chaudes/' + image ,cv2.IMREAD_GRAYSCALE)
		#kp,des = sift.detectAndCompute(img,None)
		kp = sift.detect(img,mask = mask)
		kp,des = sift.compute(img,kp)
		descripteurs = des
		
	else:
		img_th = cv2.imread('./cv2_thermiques/' + nom_csv + '.png',cv2.IMREAD_GRAYSCALE)
		mask_th = np.load('./mask_thermique/'+ nom_csv + '.npy')
		
		#kp,des = sift.detectAndCompute(img,None)
		kp_th = sift_th.detect(img_th,mask = mask_th)
		kp_th,des_th = sift_th.compute(img_th,kp_th)		
		descripteurs_th = np.concatenate((descripteurs_th,des_th),axis=0)
		
		img = cv2.imread('./Photographies pieces chaudes/' + image ,cv2.IMREAD_GRAYSCALE)
		#kp,des = sift.detectAndCompute(img,None)
		kp = sift.detect(img,mask = mask)
		kp,des = sift.compute(img,kp)		
		descripteurs = np.concatenate((descripteurs,des),axis=0)
		
	i+=1
	print(i)

descripteurs_th = np.float32(descripteurs_th)
descripteurs = np.float32(descripteurs)
print(descripteurs_th.shape)
print(descripteurs.shape)
np.save('descripteurs_th.npy',descripteurs_th)
np.save('descripteurs.npy',descripteurs)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
(650, 128)
(5309, 128)


Création des labels

In [20]:
#Creation des labels (tache : non conforme (1) si q_tache>3)
q_tache_train = df_train['q_coulure'].fillna(0)
Y_train = q_tache_train.to_numpy(np.float32)
Y_train = (Y_train >3)
Y_train = np.int16(Y_train)

q_tache_test = df_test['q_coulure'].fillna(0)
Y_test = q_tache_test.to_numpy(np.float32)
Y_test = (Y_test >3)
Y_test = np.int16(Y_test)


np.save('Y_train.npy',Y_train)
np.save('Y_test.npy',Y_test)

## Création des histogrammes sur les jeux  
Sur les données visibles

In [21]:

descripteurs = np.load('descripteurs.npy')
descripteurs_th = np.load('descripteurs_th.npy')

#Calculs des center des k means
nbre_clusters = 500

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
ret,label,center=cv2.kmeans(descripteurs,nbre_clusters,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
np.save('center_k_means.npy',center)

Y_train = np.load('Y_train.npy')
n = Y_train.shape[0]
X_train = np.zeros((n,nbre_clusters))

center_k_means = np.load('center_k_means.npy')

k=0
for indice,image in zip(indices_train,images_train):	
	img = cv2.imread('./Photographies pieces chaudes/' + image ,cv2.IMREAD_GRAYSCALE)
	#kp,des = sift.detectAndCompute(img,None)
	kp = sift.detect(img,mask = mask)
	kp,des = sift.compute(img,kp)
	nbr_des = des.shape[0]
	x = np.zeros((nbre_clusters,))
	for elem in des:
		distance = np.linalg.norm(center_k_means - elem,axis=1)
		j = np.argmin(distance)
		x[j]+=1
	x = x/nbr_des
	X_train[k] = x
	k+=1
	print(k)

np.save('X_train_vi.npy',X_train)
print(X_train.shape)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
(36, 500)


In [22]:
m = len(images)-X_train.shape[0]
X_test = np.zeros((m,nbre_clusters))


k=0
for indice,image in zip(indices_test,images_test):	

	img = cv2.imread('./Photographies pieces chaudes/' + image ,cv2.IMREAD_GRAYSCALE)
	#kp,des = sift.detectAndCompute(img,None)
	kp = sift.detect(img,mask = mask)
	kp,des = sift.compute(img,kp)
	nbr_des = des.shape[0]
	x = np.zeros((nbre_clusters,))
	for elem in des:
		distance = np.linalg.norm(center_k_means - elem,axis=1)
		j = np.argmin(distance)
		x[j]+=1
	x = x/nbr_des
	X_test[k] = x
	k+=1
	print(k)
	
np.save('X_test_vi.npy',X_test)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143


Et sur les données IR :

In [23]:
descripteurs_th = np.load('descripteurs_th.npy')

#Calculs des center des k means
nbre_clusters_th = 5 #tres petit nombre car faible nombre de features de base sur les img thermiques

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
ret,label,center_th=cv2.kmeans(descripteurs_th,nbre_clusters_th,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
np.save('center_k_means_th.npy',center_th)

Y_train = np.load('Y_train.npy')
n = Y_train.shape[0]
X_train_th = np.zeros((n,nbre_clusters_th +2))

arr_min = np.load('./array_min_max/array_min.npy')
arr_max = np.load('./array_min_max/array_max.npy')

center_k_means_th = np.load('center_k_means_th.npy')

k=0
for indice,csv in zip(indices_train , csv_thermiques_train) :
	nom_csv = (csv.split('.'))[0]
	
	img_th = cv2.imread('./cv2_thermiques/' + nom_csv + '.png',cv2.IMREAD_GRAYSCALE)
	mask_th = np.load('./mask_thermique/'+ nom_csv + '.npy')
		
	#kp,des = sift.detectAndCompute(img,None)
	kp_th = sift_th.detect(img_th,mask = mask_th)
	kp_th,des_th = sift_th.compute(img_th,kp_th)
	nbr_des = des_th.shape[0]
	x = np.zeros((nbre_clusters_th + 2,))
	for elem in des_th:
		distance = np.linalg.norm(center_k_means_th - elem,axis=1)
		j = np.argmin(distance)
		x[j]+=1
	x = x/nbr_des
	x[nbre_clusters_th] = arr_min[indice]
	x[nbre_clusters_th+1] = arr_max[indice]
	X_train_th[k] = x
	k+=1
	print(k)

np.save('X_train_th.npy',X_train_th)
print(X_train_th.shape)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
(36, 7)


In [24]:
m = len(csv_thermiques)-X_train_th.shape[0]
X_test_th = np.zeros((m,nbre_clusters_th+2))



k=0
for indice,csv in zip(indices_test , csv_thermiques_test) :
	
	nom_csv = (csv.split('.'))[0]
	img_th = cv2.imread('./cv2_thermiques/' + nom_csv + '.png',cv2.IMREAD_GRAYSCALE)
	mask_th = np.load('./mask_thermique/'+ nom_csv + '.npy')
	
	#kp,des = sift.detectAndCompute(img,None)
	kp_th = sift_th.detect(img_th,mask = mask_th)
	kp_th,des_th = sift_th.compute(img_th,kp_th)
	nbr_des = des_th.shape[0]
	x = np.zeros((nbre_clusters_th + 2,))
	for elem in des_th:
		distance = np.linalg.norm(center_k_means_th - elem,axis=1)
		j = np.argmin(distance)
		x[j]+=1
	x = x/nbr_des
	x[nbre_clusters_th] = arr_min[indice]
	x[nbre_clusters_th +1] = arr_max[indice]
	X_test_th[k] = x
	k+=1
	print(k)
	
np.save('X_test_th.npy',X_test_th)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143


Assemblage des X :

In [25]:
X_train_vi = np.load('X_train_vi.npy')
X_test_vi = np.load('X_test_vi.npy')
X_train_th = np.load('X_train_th.npy')
X_test_th = np.load('X_test_th.npy')


X_train = np.concatenate((X_train_vi,X_train_th),axis=1)
X_test = np.concatenate((X_test_vi,X_test_th),axis=1)

print(X_train.shape)
print(X_test.shape)

np.save('X_train.npy',X_train)
np.save('X_test.npy',X_test)

(36, 507)
(143, 507)


Classifieurs  
LDA

In [26]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

LDA = LinearDiscriminantAnalysis()

X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')


LDA.fit(X_train,Y_train)
Y_pred = LDA.predict(X_test)

def matrice_confusion(pred,lab):
	#calcul des vrais positifs
	vp = np.count_nonzero((pred == 1) & (lab == 1))
	#calculs des faux positifs
	fp = np.count_nonzero((pred == 1) & (lab == 0))
	#calculs des vrais négatifs
	vn = np.count_nonzero((pred == 0) & (lab == 0))
	#calculs des faux négatifs
	fn = np.count_nonzero((pred == 0) & (lab == 1))
	return [vp,fp,vn,fn]
	
conf = matrice_confusion(Y_pred,Y_test)
print(conf)
taux_fn = conf[3]/sum(conf)
taux_fn_neg = conf[3]/(conf[3]+conf[0])
non_conf = conf[0]+conf[3]
conf = conf[1]+conf[2]

score_random = max(conf/(conf+non_conf),non_conf/(conf+non_conf))

print(f"Non conformes : {non_conf}")
print(f"Conformes : {conf}")
print(f"taux de faux conformes par rapport aux non conformes : {taux_fn_neg}")

print(f"score = {LDA.score(X_test,Y_test)}")
print(f"score du random = {score_random}")

[0, 0, 115, 28]
Non conformes : 28
Conformes : 115
taux de faux conformes par rapport aux non conformes : 1.0
score = 0.8041958041958042
score du random = 0.8041958041958042


1-NN

In [31]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)

X_train = np.load('X_train.npy')
Y_train = np.load('Y_train.npy')
X_test = np.load('X_test.npy')
Y_test = np.load('Y_test.npy')

neigh.fit(X_train,Y_train)
Y_pred = neigh.predict(X_test)

conf = matrice_confusion(Y_pred,Y_test)
print(conf)
taux_fn = conf[3]/sum(conf)
taux_fn_neg = conf[3]/(conf[3]+conf[0])
non_conf = conf[0]+conf[3]
conf = conf[1]+conf[2]

score_random = max(conf/(conf+non_conf),non_conf/(conf+non_conf))

print(f"Non conformes : {non_conf}")
print(f"Conformes : {conf}")
print(f"taux de faux conformes par rapport aux non conformes : {taux_fn_neg}")

print(f"score = {neigh.score(X_test,Y_test)}")
print(f"score du random = {score_random}")

[8, 0, 115, 20]
Non conformes : 28
Conformes : 115
taux de faux conformes par rapport aux non conformes : 0.7142857142857143
score = 0.8601398601398601
score du random = 0.8041958041958042


## Résultats

### X_train = 1 sur 5, X_test = le reste  
#### 1-NN :  
[24, 1, 114, 4]  
Non conformes : 28  
Conformes : 115  
taux de faux conformes par rapport aux non conformes : 0.14285714285714285  
score = 0.965034965034965    
score du random = 0.8041958041958042  


#### 2-NN : 
[12, 1, 114, 16]  
Non conformes : 28  
Conformes : 115  
taux de faux conformes par rapport aux non conformes : 0.5714285714285714  
score = 0.8811188811188811  
score du random = 0.8041958041958042  
#### 3-NN :  
[12, 1, 114, 16]
Non conformes : 28  
Conformes : 115  
taux de faux conformes par rapport aux non conformes : 0.5714285714285714  
score = 0.8811188811188811  
score du random = 0.8041958041958042  
#### 4-NN :  
[0, 0, 115, 28]  
Non conformes : 28  
Conformes : 115  
taux de faux conformes par rapport aux non conformes : 1.0  
score = 0.8041958041958042  
score du random = 0.8041958041958042  
#### 5-NN :  
[8, 0, 115, 20]  
Non conformes : 28  
Conformes : 115  
taux de faux conformes par rapport aux non conformes : 0.7142857142857143  
score = 0.8601398601398601   
score du random = 0.8041958041958042  


### Création de jeux qui séparent les différents essais:  
num_piece pair : test  
num_piece impair : train

#### 1-NN: 
 
#### 2-NN:  
 
#### 3-NN:  

#### 4-NN:

#### 5-NN:  
 


### Création de jeux random 60% train, 40% test

#### 1-NN:  
[11, 4, 55, 2]  
Non conformes : 13  
Conformes : 59  
taux de faux conformes par rapport aux non conformes : 0.15384615384615385  
score = 0.9166666666666666  
score du random = 0.8194444444444444   
#### 2-NN:  
[11, 0, 59, 2]   
Non conformes : 13  
Conformes : 59  
taux de faux conformes par rapport aux non conformes : 0.15384615384615385  
score = 0.9722222222222222  
score du random = 0.8194444444444444  
#### 3-NN:  
[13, 0, 59, 0]  
Non conformes : 13  
Conformes : 59  
taux de faux conformes par rapport aux non conformes : 0.0  
score = 1.0  
score du random = 0.8194444444444444  
#### 4-NN:  
[12, 0, 59, 1]
Non conformes : 13  
Conformes : 59  
taux de faux conformes par rapport aux non conformes : 0.07692307692307693  
score = 0.9861111111111112  
score du random = 0.8194444444444444  
#### 5-NN:
[12, 0, 59, 1]   
Non conformes : 13  
Conformes : 59  
taux de faux conformes par rapport aux non conformes : 0.07692307692307693  
score = 0.9861111111111112  
score du random = 0.8194444444444444  
 
  